In [1]:
from langchain import PromptTemplate #用于 PromptTemplate 为字符串提示创建模板
from langchain.prompts.pipeline import PipelinePromptTemplate #for multiple prompt merging

#base class of all prompt generators, provide single prompt given arguments
class PromptGenerator:
    def __init__(self, promptFString):
        self.prompt_template = PromptTemplate.from_template(
            promptFString
            #Sample: "Tell me a {adjective} joke about {content}."
        )
    
    def generatePrompt(self,**kwargs):
        formatted_prompt = self.prompt_template.format(**kwargs)
        return formatted_prompt


#prompt generator for MAD situation:
class MADPromptGenerator:
    def __init__(self, prePromptGuide, postPromptGuide, numOfAgents):
        self.prePromptGuide = PromptTemplate.from_template( prePromptGuide)
        self.postPromptGuide = PromptTemplate.from_template(postPromptGuide)
        self.numOfAgents = numOfAgents
        self.angentsAnsGuide = PromptTemplate.from_template("\n\n".join( [f" One agent response:{{example{index}}}" for index in range(numOfAgents)]))

    def generatePrompt(self, prevAgentsInputs, currentAgentIndex,currentRound, preInputs=None,postInputs=None):
        if preInputs !=None:
            prePrompt = self.prePromptGuide.format(preInputs)
        else:
            prePrompt =  self.prePromptGuide.format()
        
        if postInputs != None:
            postPrompt = self.postPromptGuide.format(postInputs)
        else:
            postPrompt = self.postPromptGuide.format()
        
        
        fullPrompt = prePrompt + "\n \n"
        for i in range(currentRound):
            currentAgentInput = prevAgentsInputs[i]["example{}".format(currentAgentIndex)]
            fullPrompt = fullPrompt +  f" {currentAgentInput}" + "\n\n"
            prevAgentsInputs[i]["example{}".format(currentAgentIndex)] = " "
            currentRoundAgentsContext = self.angentsAnsGuide.format(**(prevAgentsInputs[i]))
            fullPrompt = fullPrompt + "These are the recent/updated opinions from other agents:"+ currentRoundAgentsContext + "\n \n"
            prevAgentsInputs[i]["example{}".format(currentAgentIndex)] = currentAgentInput
        
        fullPrompt += postPrompt

        return fullPrompt
    

In [20]:
#return type definition
from pydantic import BaseModel, Field

class MathAns(BaseModel):
    DetailedExplainationsToConvinceOthers: str = Field(description="detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others")
    answer: float = Field(description="Final Answer")
        
class MathAnsRefinement(BaseModel):
    DetailedExplainationsToConvinceOthers: str = Field(description="detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others")
    isCorrect: bool = Field(description="True if everything in the input is correct, false something in the input is incorrect")
    answer: float = Field(description="Final Answer")

class MathAnsNoStrategy(BaseModel):
    Record: str = Field(description="everything before final answer")
    answer: float = Field(description="Final Answer")

In [34]:
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

import time

openaiApiKey = ''
model_name ="gpt-3.5-turbo-0301" #"gpt-3.5-turbo-instruct"#"gpt-3.5-turbo"# "text-davinci-003"

class ProcessHandler:
    def __init__(self,promptFString):
        self.pG = PromptGenerator(promptFString)
        self.llm = OpenAI(temperature=0.1, openai_api_key=openaiApiKey,model_name=model_name)
    
    def generateAnswer(self, questionPromptString):
        formatedQuestion = self.pG.generatePrompt(questionPromptString)
        return self.llm(formatedQuestion)
    
class MADProcessHandler:
    def __init__(self,nrounds,nAgents, prePromptString, postPromptString, parser):
        self.pG = MADPromptGenerator(prePromptString,postPromptString,nAgents)
        self.nrounds = nrounds
        self.llm = OpenAI( openai_api_key=openaiApiKey,model_name=model_name)
        self.answersRecords = []
        self.nAgents = nAgents
        self.parser = parser

    def generateAnswer(self, questionSting, prePromptString=None, postPromptString=None):
        self.answersRecords.clear()
        if prePromptString != None:
            questionSting += prePromptString
        for i in range(self.nrounds):
            self.answersRecords.append({})
            for j in range(self.nAgents):
                prompt = questionSting
                if i !=0:
                    print(i)
                    prompt = prompt + self.pG.generatePrompt(self.answersRecords, j,i)
                prompt = prompt+self.parser.get_format_instructions()
                #print("round{}".format(i), "agent{} prompt: \n".format(j),prompt)
                answer = self.llm(prompt)
                #print("round{}".format(i), "agent{} answer: \n".format(j),answer)
                time.sleep(30)
                self.answersRecords[i]["example{}".format(j)] = answer
        
        prompt = questionSting+"\n\n" + "The following paragraphs are previous answers given by other agents for your reference \n\n"
        for j in range(self.nAgents):
            prompt = prompt+ "agent{}'s answer:\n".format(j)+self.answersRecords[-1]["example{}".format(j)]+"\n\n"
        prompt = prompt + "Given the answers from other agents, please draw a final conclusion. Please Think Step by Step. Make sure to state your answer at the end of the response."

        prompt = prompt+"\n"+self.parser.get_format_instructions()
        answer = self.llm(prompt)
        
        answer = self.parser.parse(answer)
        return answer

In [36]:

class MADProcessHandlerOnlyFinalParse:
    def __init__(self,nrounds,nAgents, prePromptString, postPromptString, parser):
        self.pG = MADPromptGenerator(prePromptString,postPromptString,nAgents)
        self.nrounds = nrounds
        self.llm = OpenAI( openai_api_key=openaiApiKey,model_name=model_name)
        self.answersRecords = []
        self.nAgents = nAgents
        self.parser = parser

    def generateAnswer(self, questionSting, prePromptString=None, postPromptString=None):
        self.answersRecords.clear()
        if prePromptString != None:
            questionSting += prePromptString
        for i in range(self.nrounds):
            self.answersRecords.append({})
            for j in range(self.nAgents):
                prompt = questionSting
                if i !=0:
                    print(i)
                    prompt = prompt + self.pG.generatePrompt(self.answersRecords, j,i)
                prompt = prompt
                #print("round{}".format(i), "agent{} prompt: \n".format(j),prompt)
                answer = self.llm(prompt)
                #print("round{}".format(i), "agent{} answer: \n".format(j),answer)
                time.sleep(30)
                self.answersRecords[i]["example{}".format(j)] = answer
        
        prompt = questionSting+"\n\n" + "The following paragraphs are previous answers given by other agents for your reference \n\n"
        for j in range(self.nAgents):
            prompt = prompt+ "agent{}'s answer:\n".format(j)+self.answersRecords[-1]["example{}".format(j)]+"\n\n"
        prompt = prompt + "Given the answers from other agents, please draw a final conclusion. Please Think Step by Step. Make sure to state your answer at the end of the response."

        prompt = prompt+"\n"+self.parser.get_format_instructions()
        answer = self.llm(prompt)
        
        answer = self.parser.parse(answer)
        return answer

In [25]:
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

import time

openaiApiKey = ''
model_name ="gpt-3.5-turbo-0301" #"gpt-3.5-turbo-instruct"#"gpt-3.5-turbo"# "text-davinci-003"

class SelfRefinementProcessHandler:
    def __init__(self,questionPromptString,refinePromptFString,questionParser, refinementParser, maxAttempt):
        self.qG = PromptGenerator(questionPromptString)
        self.qP = questionParser
        self.rG = PromptGenerator(refinePromptFString)
        self.rP = refinementParser
        self.llm = OpenAI(openai_api_key=openaiApiKey,model_name=model_name)
        self.maxAttempt = maxAttempt
        self.previousAns = None
    
    def generateAnswer(self, questionPromptString):
        answer = self.llm(self.qG.generatePrompt(**{"questionString":questionPromptString}) + self.qP.get_format_instructions())
        #print("First Prompt:\n",self.qG.generatePrompt(**{"questionString":questionPromptString}) + self.qP.get_format_instructions()
        #      ,'\n',"First Ans: \n",answer)
        self.previousAns = answer
        time.sleep(30)
        y = 0 
        while(y<self.maxAttempt):
            #print(y)
            answer = self.llm(self.rG.generatePrompt(**{"previousAnswer":self.previousAns}) + self.rP.get_format_instructions()+"\n only json format output should be given")
            #print("{}th Prompt:\n".format(y),self.rG.generatePrompt(**{"previousAnswer":self.previousAns}) + self.rP.get_format_instructions()+"\n only json format output should be given"
            #      ,'\n',"{}th Ans: \n".format(y),answer)
            ans = self.rP.parse(answer)
            if(ans.isCorrect):
                self.previousAns = answer  
                break
            self.previousAns = answer   
            
            time.sleep(30)
            y+=1
        return self.rP.parse(self.previousAns)

# Testing Part

### MAD Test

In [35]:
import numpy as np
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

agents = 2
rounds = 2

parser = PydanticOutputParser(pydantic_object=MathAns)
ph = MADProcessHandler(rounds,agents,
     " Make sure to state your answer at the end of the response."
     ,"Use these opinions carefully as additional advice, can you provide an updated answer with reason? Make sure to state your answer at the end of the response.",
    parser)

a, b, c, d, e, f, g, h, i = np.random.randint(-30, 30, size=9)


answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
question_prompt = "What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})?Please Think Step by Step and given detailed explaination for each step to convince others, the explainations should be as detailed and comprehensive as possible.".format(a, b, c, d, e, f,g,h,i)

print("Final Answer:",(ph.generateAnswer(question_prompt)).answer,"Ground truth:",answer)

C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


1


KeyboardInterrupt: 

### Self RefinementTest

In [13]:

import numpy as np
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

maxAttempt = 2

ansParser = PydanticOutputParser(pydantic_object=MathAns)
refinementParser = PydanticOutputParser(pydantic_object=MathAnsRefinement)

ph = SelfRefinementProcessHandler(
     "{questionString}",
     "{previousAnswer} \n There is an error in the solution above because of lack of understanding of the question. What is the error? To find the error, go through semantically complete blocks of the solution, and check if everything looks good.",
    ansParser,
    refinementParser,
    maxAttempt
    )

a, b, c, d, e, f, g, h, i = np.random.randint(-30, 30, size=9)


answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
question_prompt = "What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})?".format(a, b, c, d, e, f,g,h,i)

print("Final Answer:",(ph.generateAnswer(question_prompt)).answer,"Ground truth:",answer)

C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


First Prompt:
 What is the result of (-8)+(-26)*(22)+(-20)-(22)*(12)+(-4)*(-6)/(25)?The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"DetailedExplainationsToConvinceOthers": {"title": "Detailedexplainationstoconvinceothers", "description": "detailed record of explainations, should be as detailed and lengthy (at least 100 words) as possible in order to convince others", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvinceOthers", "answer"]}
``` 
 First Ans: 
 {"DetailedExplainationsToC

In [17]:
#no strategy test
np.random.seed(2023)

questions = []
for _ in range(100):
    questions.append(np.random.randint(-30, 30, size=9))
    #print(questions[_])
n = 0

parser = PydanticOutputParser(pydantic_object=MathAnsNoStrategy)
llm = OpenAI( openai_api_key=openaiApiKey,model_name=model_name)
correct = 0
while(n<20):
    print(n,"/","100", "correct:", correct)
    a, b, c, d, e, f, g, h, i = questions[n]
    prompt = "What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i)+ parser.get_format_instructions()
    ans = llm(prompt)
    answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
    try:
        ans = parser.parse(ans)
        if abs(ans.answer - answer)<0.01:
            correct+=1
        print("The result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i),"gt:",answer,"gpt3.5:",ans.answer)
    except Exception as e:
        print(e)
        n-=1
    time.sleep(30)
    n=n+1

print(n,"/","100", "correct:", correct)


C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


0 / 100 correct: 0
The result of (-7)+(-5)*(-24)+(25)-(3)*(-2)+(-27)*(22)/(22)?  gt: 117.0 gpt3.5: 46.0
1 / 100 correct: 0
The result of (-8)+(7)*(2)+(-8)-(-13)*(17)+(23)*(17)/(-17)?  gt: 196.0 gpt3.5: 148.0
2 / 100 correct: 0
The result of (9)+(-25)*(-6)+(20)-(-27)*(28)+(-2)*(-13)/(16)?  gt: 936.625 gpt3.5: 769.625
3 / 100 correct: 0
The result of (18)+(-23)*(19)+(18)-(28)*(29)+(2)*(-1)/(19)?  gt: -1213.1052631578948 gpt3.5: -917.0
4 / 100 correct: 0
The result of (10)+(19)*(20)+(13)-(19)*(18)+(19)*(23)/(8)?  gt: 115.625 gpt3.5: 225.0
5 / 100 correct: 0
The result of (22)+(-24)*(-15)+(17)-(-6)*(-22)+(17)*(6)/(-29)?  gt: 263.48275862068965 gpt3.5: -504.0689655172414
6 / 100 correct: 0
The result of (-23)+(-8)*(23)+(-23)-(15)*(22)+(-5)*(4)/(19)?  gt: -561.0526315789474 gpt3.5: -102.84210526315789
7 / 100 correct: 0
The result of (-17)+(18)*(24)+(-3)-(22)*(21)+(21)*(-7)/(15)?  gt: -59.8 gpt3.5: -303.8
8 / 100 correct: 0
The result of (4)+(10)*(8)+(6)-(4)*(-19)+(16)*(-29)/(1)?  gt: -298.0

In [21]:
import numpy as np

np.random.seed(2023)

questions = []
for _ in range(100):
    questions.append(np.random.randint(-30, 30, size=9))
    #print(questions[_])
n = 0

parser = PydanticOutputParser(pydantic_object=MathAns)
llm = OpenAI(openai_api_key=openaiApiKey,model_name=model_name)
correct = 0
while(n<20):
    print(n,"/","20", "correct:", correct)
    a, b, c, d, e, f, g, h, i = questions[n]
    prompt = "\n"+"What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? Please think step by step.".format(a, b, c, d, e, f,g,h,i)+parser.get_format_instructions()
    if n==0:
        print(prompt)
    ans = llm(prompt)
    if n==0:
        print(ans)
    answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
    try:
        ans = parser.parse(ans)
        if abs(ans.answer - answer)<0.01:
            correct+=1
        print("The result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i),"gt:",answer,"gpt3.5:",ans.answer)
    except Exception as e:
        print(e)
        n-=1
    time.sleep(30)
    n=n+1

print(n,"/","100", "correct:", correct)

C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


0 / 20 correct: 0

What is the result of (-7)+(-5)*(-24)+(25)-(3)*(-2)+(-27)*(22)/(22)? Please think step by step.The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"DetailedExplainationsToConvinceOthers": {"title": "Detailedexplainationstoconvinceothers", "description": "detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvinceOthers", "answer"]}
```
{
  

In [26]:
import numpy as np

np.random.seed(2023)

questions = []
for _ in range(100):
    questions.append(np.random.randint(-30, 30, size=9))
    #print(questions[_])
n = 0

maxAttempt = 2

ansParser = PydanticOutputParser(pydantic_object=MathAns)
refinementParser = PydanticOutputParser(pydantic_object=MathAnsRefinement)

ph = SelfRefinementProcessHandler(
     "{questionString}",
     "{previousAnswer} \n There is an error in the solution above because of lack of understanding of the question. What is the error? To find the error, go through semantically complete blocks of the solution, and check if everything looks good.",
    ansParser,
    refinementParser,
    maxAttempt
    )


correct = 0
while(n<20):
    print(n,"/","20", "correct:", correct)
    a, b, c, d, e, f, g, h, i = questions[n]
    prompt = "\n"+"What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? Please think step by step.".format(a, b, c, d, e, f,g,h,i)+parser.get_format_instructions()
   
    answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
    try:
        ans = ph.generateAnswer(prompt)
        if abs(ans.answer - answer)<0.01:
            correct+=1
        print("The result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i),"gt:",answer,"gpt3.5:",ans.answer)
    except Exception as e:
        print(e)
        n-=1
    time.sleep(30)
    n=n+1

print(n,"/","100", "correct:", correct)

C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


0 / 20 correct: 0
Failed to parse MathAnsRefinement from completion {
  "DetailedExplainationsToConvicneOthers": "The error in the given solution is that the order of operations used is wrong. The correct order of operations is PEMDAS, which stands for Parentheses, Exponents, Multiplication and Division (from left to right), and Addition and Subtraction (from left to right). Therefore, we need to perform (-5)*(-24) = 120, (-27)*(22)/(22) = -27, (-3)*(-2) = 6, and then substitute these values to get the expression: (-7) + 120 + 25 - 6 - 27 = 105 - 33 = -28. Hence, the solution provided is incorrect.\n\nTo arrive at the correct answer, we will follow the correct order of operations as stated above. First, we will perform the operations within parentheses, but since there are no parentheses given, we will move to exponents, but there are none. Next, we will perform multiplication and division from left to right.\n\nStarting with multiplication, -5 * -24 = 120. Next, proceeding with divisi

6 / 20 correct: 0
Failed to parse MathAnsRefinement from completion {
  "DetailedExplainationsToConvinceOthers": "The solution provided is incorrect because it misinterprets the question. The question asks for the value of -(23 + 8 × 23 - 15 × 22 + 5 × 4/19) and not -(23) + 8 × 23 - 15 × 22 + 5 × 4/19 as the solution assumes. Using the correct order of operations (PEMDAS), we first simplify the parentheses to get -(23+184-330+0.2105). Then, we perform addition and subtraction from left to right to get -538.2105 as the final answer.",
  "isCorrect": false,
  "answer": null
}. Got: 1 validation error for MathAnsRefinement
answer
  none is not an allowed value (type=type_error.none.not_allowed)
6 / 20 correct: 0
The result of (-23)+(-8)*(23)+(-23)-(15)*(22)+(-5)*(4)/(19)?  gt: -561.0526315789474 gpt3.5: -385.895
7 / 20 correct: 0
The result of (-17)+(18)*(24)+(-3)-(22)*(21)+(21)*(-7)/(15)?  gt: -59.8 gpt3.5: 962.2
8 / 20 correct: 0
The result of (4)+(10)*(8)+(6)-(4)*(-19)+(16)*(-29)/(1)? 

In [37]:
import numpy as np
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

agents = 2
rounds = 2

parser = PydanticOutputParser(pydantic_object=MathAns)
ph = MADProcessHandler(rounds,agents,
     " Make sure to state your answer at the end of the response."
     ,"Use these opinions carefully as additional advice, can you provide an updated answer? Make sure to state your answer at the end of the response.",
    parser)

a, b, c, d, e, f, g, h, i = np.random.randint(-30, 30, size=9)
n=0
correct = 0
while(n<20):
    print(n,"/","20", "correct:", correct)
    a, b, c, d, e, f, g, h, i = questions[n]
    question_prompt = "What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? Please think step by step.".format(a, b, c, d, e, f,g,h,i)
    try:
        ans = ph.generateAnswer(question_prompt)
        answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
        if abs(ans.answer - answer)<0.01:
            correct+=1
        print("The result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i),"gt:",answer,"gpt3.5:",ans.answer)
    except Exception as e:
        print(e)
        n-=1
    time.sleep(30)
    n=n+1
print(n,"/","100", "correct:", correct)


C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


0 / 20 correct: 0
1
1
Failed to parse MathAns from completion The correct answer to the expression (-7)+(-5)*(-24)+(25)-(3)*(-2)+(-27)*(22)/(22) is 117. 

To arrive at this answer, we need to use the order of operations (PEMDAS) to simplify the expression step-by-step. 

Starting with the multiplication and division, we have (-5) * (-24) = 120, and (-27)*(22)/(22) = -27. So the expression becomes (-7) + 120 + 25 - (3*-2) - 594/22. 

Next, we simplify the parentheses: (3 * -2) = -6. And 594/22 = 27. 

Finally, we add and subtract left to right: -7 + 120 = 113. 113 + 25 = 138. 138 - (-6) = 144. And 144 - 27 = 117. 

Therefore, the final answer is 117.

{
  "properties": {
    "DetailedExplainationsToConvinceOthers": {
      "title": "Detailedexplainationstoconvinceothers",
      "description": "detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others",
      "type": "string"
    },
    "answer": {
      "title

KeyboardInterrupt: 

In [38]:
import numpy as np
from langchain.output_parsers import PydanticOutputParser, RetryWithErrorOutputParser

agents = 2
rounds = 2

parser = PydanticOutputParser(pydantic_object=MathAns)
ph = MADProcessHandlerOnlyFinalParse(rounds,agents,
     " Make sure to state your answer at the end of the response."
     ,"Use these opinions carefully as additional advice, can you provide an updated answer? Make sure to state your answer at the end of the response.",
    parser)

a, b, c, d, e, f, g, h, i = np.random.randint(-30, 30, size=9)
n=0
correct = 0
while(n<20):
    print(n,"/","20", "correct:", correct)
    a, b, c, d, e, f, g, h, i = questions[n]
    question_prompt = "What is the result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? Please think step by step.".format(a, b, c, d, e, f,g,h,i)
    try:
        ans = ph.generateAnswer(question_prompt)
        answer = (a) + (b) * (c) + (d) - (e) * (f)+ (g)*(h)/(i)
        if abs(ans.answer - answer)<0.01:
            correct+=1
        print("The result of ({})+({})*({})+({})-({})*({})+({})*({})/({})? ".format(a, b, c, d, e, f,g,h,i),"gt:",answer,"gpt3.5:",ans.answer)
    except Exception as e:
        print(e)
        n-=1
    time.sleep(30)
    n=n+1
print(n,"/","100", "correct:", correct)

C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\30477\anaconda3\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


0 / 20 correct: 0
1
1
The result of (-7)+(-5)*(-24)+(25)-(3)*(-2)+(-27)*(22)/(22)?  gt: 117.0 gpt3.5: 117.0
1 / 20 correct: 1
1
1
Failed to parse MathAns from completion {
  "DetailedExplainationsToConvincOthers": "To solve this problem, we need to follow the order of operations (PEMDAS). First, we calculate the multiplication and division operations. In this case, we have (7)*(2) = 14, (-13)*(17) = -221, (23)*(17) = 391, and (23)*(17)/(-17) = -23. Next, we replace these values in the original equation to get (-8) + 14 + (-8) - (-221) + (-23). Finally, we simplify by adding and subtracting the resulting terms to get the final answer, which is 196. Therefore, the result of (-8) + (7)*(2) + (-8) - (-13)*(17) + (23)*(17)/(-17) is 196.",
  "answer": 196
}. Got: 1 validation error for MathAns
DetailedExplainationsToConvinceOthers
  field required (type=value_error.missing)
1 / 20 correct: 1
1
1
The result of (-8)+(7)*(2)+(-8)-(-13)*(17)+(23)*(17)/(-17)?  gt: 196.0 gpt3.5: -310.0
2 / 20 corr

5 / 20 correct: 2
1
1
The result of (22)+(-24)*(-15)+(17)-(-6)*(-22)+(17)*(6)/(-29)?  gt: 263.48275862068965 gpt3.5: -56.724137931034484
6 / 20 correct: 2
1
1
Failed to parse MathAns from completion {"properties": {"DetailedExplainationsToConvinceOthers": {"title": "Detailedexplainationstoconvinceothers", "description": "detailed record of explanations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvinceOthers", "answer"]}

To solve this expression, we need to follow the order of operations, which is parentheses, exponents, multiplication/division (performed left to right), and addition/subtraction (performed left to right).

 The given expression is:
(-23)+(-8)*(23)+(-23)-(15)*(22)+(-5)*(4)/(19)

Using the order of operations, we start by simplifying the multiplication and division from lef

10 / 20 correct: 3
1
1
The result of (-25)+(-20)*(-23)+(-11)-(25)*(17)+(-24)*(25)/(27)?  gt: -23.22222222222222 gpt3.5: -1.22
11 / 20 correct: 3
1
1
The result of (5)+(-2)*(21)+(16)-(-16)*(-24)+(7)*(-5)/(6)?  gt: -410.8333333333333 gpt3.5: 363.1667
12 / 20 correct: 3
1
1
Failed to parse MathAns from completion {"properties": {"DetailedExplainationsToConvinceOthers": {"title": "Detailedexplainationstoconvinceothers", "description": "detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvinceOthers", "answer"]}

Following the Order of Operations, we first evaluate the expression inside the parentheses and brackets, and then evaluate any exponents. In this case, there are no parentheses, brackets, or exponents, so we move on to multiplication and division, working from

13 / 20 correct: 4
1
1
Failed to parse MathAns from completion {"properties": {"DetailedExplainationsToConvineOthers": {"title": "Detailed explanations to convince others", "description": "detailed record of explanations should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others.", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvineOthers", "answer"]}

Following the order of operations (PEMDAS), we must first perform any calculations inside parentheses or brackets. However, there are no parentheses or brackets in this expression. Next, we should perform any exponentiations. But there are no exponentiations in this expression either.

Moving on to multiplication and division, we should perform these operations from left to right. Starting from the left:

- (-22)*(-4) = 88 (multiplying two negative numbers gives a positive result)
- (25)*(24) = 600 (regu

16 / 20 correct: 7
1
1
The result of (-30)+(12)*(10)+(1)-(9)*(-15)+(3)*(-22)/(-29)?  gt: 228.27586206896552 gpt3.5: 228.27586207
17 / 20 correct: 8
1
1
The result of (14)+(-3)*(-20)+(12)-(26)*(-30)+(-24)*(24)/(-25)?  gt: 889.04 gpt3.5: 843.0
18 / 20 correct: 8
1
1
The result of (7)+(-17)*(-19)+(28)-(1)*(-10)+(-27)*(-10)/(16)?  gt: 384.875 gpt3.5: 384.875
19 / 20 correct: 9
1
1
Failed to parse MathAns from completion {"properties": {"DetailedExplainationsToConvinceOthers": {"title": "DetailedExplainationsToConvinceOthers", "description": "detailed record of explainations, should be as detailed and lengthy (no shorter than 200 words) as possible in order to convince others", "type": "string"}, "answer": {"title": "Answer", "description": "Final Answer", "type": "number"}}, "required": ["DetailedExplainationsToConvinceOthers", "answer"]}

To solve (-18)+(16)*(14)+(27)-(-24)*(17)+(7)*(6)/(-11), we must follow the order of operations (PEMDAS). 

The problem contains parentheses, multiplicat